In [30]:
from pathlib import Path
import glob
import random
import pathlib
import music21
from music21 import converter, instrument, note, chord
import numpy as np
from keras.utils import np_utils
import pandas as pd
from tensorflow.python.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Dropout, LSTM, Activation
from tensorflow.python.keras.models import Sequential
import music21.note
from music21 import stream


In [31]:
# folder = pathlib.Path('maestro-v3.0.0-midi/maestro-v3.0.0/2018')
# filenames = glob.glob(str(folder/'*.midi'))
# print(len(filenames))

songs = []
folder = Path('maestro-v3.0.0-midi/maestro-v3.0.0/2018')
for file in folder.rglob('*.midi'):
  songs.append(file)
print(len(songs))

import random
# Get a subset of 1000 songs
result =  random.sample([x for x in songs], 10)

93


In [32]:
notes = []
for i,file in enumerate(result):
    print(f'{i+1}: {file}')
    try:
      midi = converter.parse(file)
      notes_to_parse = None
      parts = instrument.partitionByInstrument(midi)
      if parts: # file has instrument parts
          notes_to_parse = parts.parts[0].recurse()
      else: # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))
    except:
      print(f'FAILED: {i+1}: {file}')

1: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Recital9-11_MID--AUDIO_11_R1_2018_wav--1.midi
2: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Recital17-19_MID--AUDIO_19_R1_2018_wav--3.midi
3: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Recital1-3_MID--AUDIO_03_R1_2018_wav--2.midi
4: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Recital9-11_MID--AUDIO_11_R1_2018_wav--2.midi
5: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--1.midi
6: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--2.midi
7: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Recital17-19_MID--AUDIO_17_R1_2018_wav--3.midi
8: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Recital5-7_MID--AUDIO_06_R1_2018_wav--1.midi
9: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unprocessed_Recital5-7_MID--AUDIO_07_R1_2018_wav--4.midi
10: maestro-v3.0.0-midi\maestro-v3.0.0\2018\MIDI-Unpr

In [33]:
import pickle
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

In [45]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32

    # Get all unique pitchnames
    pitchnames = sorted(set(item for item in notes))
    numPitches = len(pitchnames)

     # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        # sequence_in is a sequence_length list containing sequence_length notes
        sequence_in = notes[i:i + sequence_length]
        # sequence_out is the sequence_length + 1 note that comes after all the notes in
        # sequence_in. This is so the model can read sequence_length notes before predicting
        # the next one.
        sequence_out = notes[i + sequence_length]
        # network_input is the same as sequence_in but it containes the indexes from the notes
        # because the model is only fed the indexes.
        network_input.append([note_to_int[char] for char in sequence_in])
        # network_output containes the index of the sequence_out
        network_output.append(note_to_int[sequence_out])

    # n_patters is the length of the times it was iterated 
    # for example if i = 3, then n_patterns = 3
    # because network_input is a list of lists
    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    # Reshapes it into a n_patterns by sequence_length matrix
    print(len(network_input))
    
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # OneHot encodes the network_output
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes,n_vocab)
n_patterns = len(network_input)
pitchnames = sorted(set(item for item in notes))
numPitches = len(pitchnames)

#print(network_input)

55645


In [ ]:
# def oversample(network_input,network_output,sequence_length=15):

#   n_patterns = len(network_input)
#   # Create a DataFrame from the two matrices
#   new_df = pd.concat([pd.DataFrame(network_input),pd.DataFrame(network_output)],axis=1)

#   # Rename the columns to numbers and Notes
#   new_df.columns = [x for x in range(sequence_length+1)]
#   new_df = new_df.rename(columns={sequence_length:'Notes'})

#   print(new_df.tail(20))
#   print('###################################################')
#   print(f'Distribution of notes in the preoversampled DataFrame: {new_df["Notes"].value_counts()}')
#   # Oversampling
#   oversampled_df = new_df.copy()
#   #max_class_size = np.max(oversampled_df['Notes'].value_counts())
#   max_class_size = 700
#   print('Size of biggest class: ', max_class_size)

#   class_subsets = [oversampled_df.query('Notes == ' + str(i)) for i in range(len(new_df["Notes"].unique()))] # range(2) because it is a binary class

#   for i in range(len(new_df['Notes'].unique())):
#     try:
#       class_subsets[i] = class_subsets[i].sample(max_class_size,random_state=42,replace=True)
#     except:
#       print(i)

#   oversampled_df = pd.concat(class_subsets,axis=0).sample(frac=1.0,random_state=42).reset_index(drop=True)

#   print('###################################################')
#   print(f'Distribution of notes in the oversampled DataFrame: {oversampled_df["Notes"].value_counts()}')

#   # Get a sample from the oversampled DataFrame (because it may be too big, and we also have to convert it into a 3D array for the LSTM)
#   sampled_df = oversampled_df.sample(n_patterns,replace=True) # 99968*32 has to be equals to (99968,32,1)

#   print('###################################################')
#   print(f'Distribution of notes in the oversampled post-sampled DataFrame: {sampled_df["Notes"].value_counts()}')

#   # Convert the training columns back to a 3D array
#   network_in = sampled_df[[x for x in range(sequence_length)]]
#   network_in = np.array(network_in)
#   network_in = np.reshape(network_input, (n_patterns, sequence_length, 1))
#   network_in = network_in / numPitches
#   print(network_in.shape)
#   print(sampled_df['Notes'].shape)
#   # Converts the target column into a OneHot encoded matrix
#   network_out = pd.get_dummies(sampled_df['Notes'])
#   print(network_out.shape)

#   return network_in,network_out

# networkInputShaped,networkOutputShaped = oversample(network_input,network_output,sequence_length=15)
# networkOutputShaped = np_utils.to_categorical(network_output)

In [35]:
model = Sequential()
model.add(Dropout(0.2))
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dense(256))
model.add(Dense(256))
model.add(LSTM(512, return_sequences=True))
model.add(Dense(256))
model.add(LSTM(512))
#model.add(Dense(numPitches))
model.add(Dense(numPitches))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [36]:
num_epochs = 2

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger_1.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=1,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]

history = model.fit(network_input, network_output, epochs=num_epochs, batch_size=64, callbacks=callbacks_list)


Epoch 1/2
870/870 [==============================] - 1168s 1s/step - loss: 5.2457 - accuracy: 0.0168

Epoch 00001: loss improved from inf to 5.24572, saving model to weights-improvement-01-5.2457-bigger_1.hdf5
Epoch 2/2
870/870 [==============================] - 1165s 1s/step - loss: 5.2024 - accuracy: 0.0179

Epoch 00002: loss improved from 5.24572 to 5.20243, saving model to weights-improvement-02-5.2024-bigger_1.hdf5


In [46]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    # Selects a random row from the network_input
    start = np.random.randint(0, len(network_input)-1)
    print(f'start: {start}')
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Random row from network_input
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        # Reshapes pattern into a vector
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        # Standarizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Outputs a OneHot encoded vector, so this picks the columns
        # with the highest probability
        index = np.argmax(prediction)
        # Maps the note to its respective index
        result = int_to_note[index]
        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = np.append(pattern,index)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:len(pattern)]

    return prediction_output

sequence_length = 32

n_vocab = len(set(notes))
pitchnames = sorted(set(item for item in notes))
prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)

start: 35566


In [42]:
prediction_output = ['B2', 'B2', '2.7', '5.10']

In [ ]:


prediction_input = np.reshape(pattern, (1, len(pattern), 1))

prediction = model.predict(prediction_input, verbose=0)
print(prediction)

In [47]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test.mid')
create_midi(prediction_output)

## OTHER TRY

In [2]:
from mido import MidiFile, MidiTrack, Message
import numpy as np
import os 
import pretty_midi

def midi_to_array(midi_path):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(midi_path)
    
    # Get the piano roll representation
    piano_roll = midi_data.get_piano_roll(fs=1)  # Set fs to control time resolution
    
    # Transpose the piano roll to have time steps as rows and pitches as columns
    piano_roll = piano_roll.T
    
    return piano_roll

# def midi_to_notes(midi_file, length = 1000):
#     mid = MidiFile(midi_file)
#     notes = []
#     for msg in mid:
#         if not msg.is_meta and msg.channel == 0 and msg.type == "note_on":
#             data = msg.bytes()
#             if data[2] != 0:
#                 notes.append(data[1])
#     notes = np.array(notes)
#     notes = notes[:length]
#     return notes

# Example usage
midi_path = 'maestro-v3.0.0-midi/maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1.midi'
midi_pathg = "maestro-v3.0.0-midi/maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi"
midi_array = midi_to_array(midi_path)
midi_arrayg = midi_to_array(midi_pathg)
print(midi_array.shape)  # Print the shape of the resulting numpy array
print(midi_arrayg.shape)

(582, 128)
(703, 128)


In [84]:
data = []
for file in os.listdir("maestro-v3.0.0-midi/maestro-v3.0.0/2018"):
    if file.endswith(".midi"):
        data.append(midi_to_array("maestro-v3.0.0-midi/maestro-v3.0.0/2018/" + file))

In [85]:
#pad the data
max_len = max([x.shape[0] for x in data])
max_len

2562

In [86]:
padded_data = []
for x in data:
    pad = np.zeros((max_len - x.shape[0], x.shape[1]))
    padded_data.append(np.concatenate((x, pad), axis=0))


In [87]:
padded_data[0].shape

(2562, 128)

In [93]:
#conver padded data to midi files
padded_data[0] = padded_data[0].T
padded_data[0].shape

#convert to midi
def array_to_midi(array, name):
    array = array.T
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)
    for i in range(array.shape[0]):
        if array[i, 0] != 0:
            track.append(Message('note_on', note=int(array[i, 0]), velocity=int(array[i, 1]), time=int(array[i, 2])))
    midi.save(name)

array_to_midi(data[0], "test.midi")


In [29]:
from torch.utils.data import Dataset, DataLoader
import torch

class MidiDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

    def collate_fn(self, batch):
        batch = torch.tensor(batch)
        batch = batch.transpose(0, 1)
        return batch

dataset = MidiDataset(padded_data, max_len)


In [36]:
class Generator(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.lstm = torch.nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_size)
        self.softmax = torch.nn.Softmax(dim=2)

    def forward(self, x, hidden):
        x, hidden = self.lstm(x, hidden)
        x = self.linear(x)
        x = self.softmax(x)
        return x, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(1000, batch_size, self.hidden_size), torch.zeros(1000, batch_size, self.hidden_size))

In [37]:
class Disciminator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(1, 64, 2, batch_first=True)
        self.linear = torch.nn.Linear(64, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [38]:
class GAN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GAN, self).__init__()
        self.generator = Generator(input_size, hidden_size, output_size)
        self.discriminator = Disciminator()

    def forward(self, x, hidden):
        x, hidden = self.generator(x, hidden)
        x = self.discriminator(x)
        return x, hidden

    def init_hidden(self, batch_size):
        return self.generator.init_hidden(batch_size)

def train(dataloader, model, optimizer, criterion, epochs):
    for epoch in range(epochs):
        for i, batch in enumerate(dataloader):
            batch_size = batch.shape[1]
            hidden = model.init_hidden(batch_size)
            for j in range(batch.shape[0]):
                optimizer.zero_grad()
                x = batch[j].unsqueeze(0).unsqueeze(2).float()
                y = torch.ones(batch_size, 1)
                y_hat, hidden = model(x, hidden)
                loss = criterion(y_hat, y)
                loss.backward()
                optimizer.step()
                print(loss.item())

